In [1]:
import yfinance as yf
import pandas as pd
from backtesting import Backtest, Strategy

import pandas as pd
import talib as ta

pd.set_option('display.float_format', lambda x: '%.6f' % x)

In [3]:
# dji = yf.Ticker("^DJI").history(period="max")
vix = yf.Ticker("^VIX").history(interval='1h')
vix

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Datetime,,,,,,,
2024-08-16 08:00:00-05:00,15.660000,15.670000,15.320000,15.380000,0,0.000000,0.000000
2024-08-16 09:00:00-05:00,15.560000,15.560000,14.960000,15.070000,0,0.000000,0.000000
2024-08-16 10:00:00-05:00,15.000000,15.180000,14.730000,14.760000,0,0.000000,0.000000
2024-08-16 11:00:00-05:00,14.990000,14.990000,14.650000,14.690000,0,0.000000,0.000000
2024-08-16 12:00:00-05:00,14.810000,14.920000,14.650000,14.870000,0,0.000000,0.000000
...,...,...,...,...,...,...,...
2024-09-16 04:00:00-05:00,16.969999,17.010000,16.910000,17.010000,0,0.000000,0.000000
2024-09-16 05:00:00-05:00,17.000000,17.150000,17.000000,17.040001,0,0.000000,0.000000
2024-09-16 06:00:00-05:00,17.030001,17.160000,17.030001,17.110001,0,0.000000,0.000000


In [4]:
from datetime import timedelta
import pytz
import MetaTrader5 as mt5
from datetime import datetime, timedelta


print("MetaTrader5 package author: ", mt5.__author__)
print("MetaTrader5 package version: ", mt5.__version__)

# Establecer conexión con el terminal de MetaTrader 5
if not mt5.initialize():
    raise Exception("initialize() failed, error code =", mt5.last_error())

# Establecer la zona horaria a UTC
timezone = pytz.timezone("Etc/UTC")

# Crear objetos 'datetime' en zona horaria UTC
now = datetime.now(tz=timezone)
date_from = now - timedelta(days=20) - timedelta(days=500) 
            
# Obtener las tasas históricas
rates = mt5.copy_rates_range('USTECm', mt5.TIMEFRAME_H1, date_from, now)

# Crear DataFrame con las tasas
df = pd.DataFrame(rates)

# Convertir el tiempo de segundos a formato datetime
df['time'] = pd.to_datetime(df['time'], unit='s')


# Renombrar columnas para el ticker principal
df = df.rename(columns={
    'time': 'Date',
    'open': 'Open',
    'high': 'High',
    'low': 'Low',
    'close': 'Close',
    'tick_volume': 'Volume'
}).set_index('Date')


# Cerrar la conexión con MetaTrader 5
mt5.shutdown()

MetaTrader5 package author:  MetaQuotes Ltd.
MetaTrader5 package version:  5.0.4288


True

In [5]:
vix.rename(
    columns={'Close':'VixClose'}, inplace=True
)

In [6]:
df.index = df.index.tz_localize('UTC').tz_convert('UTC')
vix.index = vix.index.tz_convert('UTC')

In [7]:
print(df.tail(1).index)
print(vix.tail(1).index)

DatetimeIndex(['2024-09-13 13:00:00+00:00'], dtype='datetime64[ns, UTC]', name='Date', freq=None)
DatetimeIndex(['2024-09-13 14:00:00+00:00'], dtype='datetime64[ns, UTC]', name='Datetime', freq=None)


In [8]:
merge = pd.merge(
    df, 
    vix[['VixClose']], 
    left_index=True, 
    right_index=True
)

merge

,Open,High,Low,Close,Volume,spread,real_volume,VixClose
2023-04-12 15:00:00+00:00,12905.330000,12976.190000,12874.650000,12969.660000,4006,358,0,19.070000
2023-04-12 16:00:00+00:00,12969.750000,12988.890000,12962.480000,12983.720000,3431,352,0,18.840000
2023-04-12 17:00:00+00:00,12984.640000,13007.110000,12956.370000,12966.530000,3076,351,0,18.559999
2023-04-12 18:00:00+00:00,12966.420000,12989.080000,12882.360000,12899.990000,4268,354,0,18.770000
2023-04-12 19:00:00+00:00,12899.150000,12915.160000,12834.640000,12848.550000,3872,385,0,19.160000
...,...,...,...,...,...,...,...,...
2024-09-13 09:00:00+00:00,19445.310000,19471.070000,19441.180000,19454.880000,2270,503,0,16.889999
2024-09-13 10:00:00+00:00,19454.470000,19464.930000,19446.130000,19460.960000,2050,504,0,16.860001
2024-09-13 11:00:00+00:00,19462.330000,19462.700000,19430.930000,19445.130000,2215,504,0,16.910000
2024-09-13 12:00:00+00:00,19444.660000,19457.280000,19403.610000,19438.590000,2742,500,0,16.889999


In [9]:
def ll_hh_indicator(close, window=None):
    if type(close) != pd.Series:
        close = close.s

    rolling_min = close.rolling(window=window, min_periods=1).min()
    is_lower_low = close == rolling_min

    rolling_max = close.rolling(window=window, min_periods=1).max()
    is_higher_high = close == rolling_max

    return is_lower_low, is_higher_high


class VixRsi(Strategy):

    def init(self):
        self.rsi = self.I(ta.RSI, self.data.Close, timeperiod=2)
        self.sma = self.I(ta.SMA, self.data.Close, timeperiod=200)

        self.vix_sma = self.I(ta.SMA, self.data.VixClose, timeperiod=10)
        self.lower_low, self.higher_high = self.I(ll_hh_indicator, self.data.Close, window=5)


    def next(self):
        actual_close = self.data.Close[-1]

        if self.position:
            first_trade = self.trades[0]
            today = self.data.index[-1].tz_convert('UTC')
            time_in_position = (today - first_trade.entry_time.tz_convert('UTC'))
            time_in_position = time_in_position.days

            if self.position.is_long:
                if self.higher_high:
                    self.position.close()

        else:
            cum_rsi = self.rsi[-1] + self.rsi[-2]

            vix_sma_value = self.vix_sma[-1]
            vix_close = self.data.VixClose[-1]
            vix_above_sma = vix_close > (vix_sma_value * (1 + 0.05))

            if vix_above_sma and cum_rsi <= 50 and actual_close > self.sma:
                self.buy(size=1)


In [10]:
# start_date = '2022-01-01'
# end_date = '2022-02-01'


bt = Backtest(
    merge, 
    VixRsi, 
    cash=100000, 
    commission=7e-4
)

stats = bt.run()


stats

Start                     2023-04-12 15:00...
End                       2024-09-13 13:00...
Duration                    519 days 22:00:00
Exposure Time [%]                    3.042434
Equity Final [$]                100607.399181
Equity Peak [$]                 100616.565196
Return [%]                           0.607399
Buy & Hold Return [%]               49.858901
Return (Ann.) [%]                    0.413288
Volatility (Ann.) [%]                0.383441
Sharpe Ratio                         1.077842
Sortino Ratio                        1.922337
Calmar Ratio                         1.476490
Max. Drawdown [%]                   -0.279913
Avg. Drawdown [%]                   -0.088728
Max. Drawdown Duration      164 days 02:00:00
Avg. Drawdown Duration       36 days 01:00:00
# Trades                                   23
Win Rate [%]                        82.608696
Best Trade [%]                       1.194868
Worst Trade [%]                     -1.054048
Avg. Trade [%]                    

In [11]:
bt.plot()

c:\Users\SaidJ\OneDrive\Documentos\projects\forex_ml_bot\forex_ml_bot\mtvenv\lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
c:\Users\SaidJ\OneDrive\Documentos\projects\forex_ml_bot\forex_ml_bot\mtvenv\lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],


GridPlot(id='p1747', ...)

In [ ]:
stats, heatmap = bt.optimize(
    upper_bound=range(50, 85, 5),
    lower_bound=range(10, 45, 5),
    rsi_window=range(10, 20, 2),
    maximize='Sharpe Ratio',
    # maximize=optim_func,
    # max_tries=100,
    return_heatmap=True
)